In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

## Walking to and fro in Julia: Benchmarking and inspecting native code

Let's write some variations on a function which computes a one-dimension random walk. We will take `n` steps, and at each step, we have the choice to move one unit left/right. We want to generate a path over `n` steps and compute the final location of the walker. We'll use Julia's in-built `Random` standard library to make the steps, and the `BenchmarkTools` package later on to determine which method is fastest.

In [1]:
function walk(T::Integer)
    x = 0
    
    for i in 1:T        
        if rand() < 0.5
            x += 1
        else
            x -= 1
        end
    end
    
    return x
end

walk (generic function with 1 method)

The `rand( (-1, +1) )` here picks one of these two options - `rand( iterable )` randomly selects a member of the collection.

In [2]:
function walk2(T::Integer)
    x = 0
    for i in 1:T
        x += rand( (-1, +1) )
    end
end

walk2 (generic function with 1 method)

In [3]:
function walk3(T::Integer)
    x = 0
    for i in 1:T
        x += (2 * convert(Integer, round(rand())) - 1)
    end
end

walk3 (generic function with 1 method)

In [4]:
function walk4(T)
    return cumsum(rand( (-1, +1), T ))[end]
end

walk4 (generic function with 1 method)

In [28]:
using Random
function walk5(T)
    return sum(2 .* bitrand(100) .- 1)
end

walk5 (generic function with 1 method)

In [8]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1273


In [29]:
@btime walk5(100)

  196.996 ns (3 allocations: 1.00 KiB)


-8

In [11]:
@btime walk4(100)

  286.768 ns (2 allocations: 1.75 KiB)


-8

In [30]:
@btime walk3(100)


  536.106 ns (0 allocations: 0 bytes)


In [32]:
@btime walk2(100)

  918.182 ns (0 allocations: 0 bytes)


In [33]:
@btime walk(100)

  583.602 ns (0 allocations: 0 bytes)


-2

In [38]:
function walk_mean(T, N)
    sumsq = 0.0
    
    for i in 1:N
        w = walk5(T)
        sumsq += w^2
    end
    
    return sumsq / N
end

walk_mean (generic function with 1 method)

In [39]:
@time walk_mean(10^4, 10^5)

  0.056225 seconds (317.26 k allocations: 98.570 MiB, 36.67% gc time)


100.26504

In [40]:
time_julia = @elapsed walk_mean(10^4, 10^5)

0.053939936

In [41]:
time_python = 113

113

In [42]:
time_python / time_julia

2094.9227674278295

In [31]:
@code_native walk3(100)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[3]:2 within `walk3'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$96, %rsp
	movq	%rdi, %r12
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 32(%rsp)
	movabsq	$jl_get_ptls_states_fast, %rax
	vzeroupper
	callq	*%rax
; │ @ In[3]:3 within `walk3'
; │┌ @ range.jl:5 within `Colon'
; ││┌ @ range.jl:277 within `UnitRange'
; │││┌ @ range.jl:282 within `unitrange_last'
; ││││┌ @ operators.jl:341 within `>='
; │││││┌ @ int.jl:424 within `<='
	movq	$4, 32(%rsp)
	movq	(%rax), %rcx
	movq	%rcx, 40(%rsp)
	leaq	32(%rsp), %rcx
	movq	%rax, 24(%rsp)
	movq	%rcx, (%rax)
	testq	%r12, %r12
; │└└└└└
	jle	L230
; │ @ In[3]:4 within `walk3'
; │┌ @ Random.jl:256 within `rand'
; ││┌ @ RNGs.jl:296 within `default_rng'
; │││┌ @ threadingconstructs.jl:10 within `threadid'
	movq	24(%rsp), %rax
	movswq	16(%rax), %r15
; │└└└
; │ @ In[3]:3 within `walk3'
	addq	$-1, %r12
	movb	$2, %r14b
	addq	$1, %r15
	

In [34]:
@code_warntype walk2(100)

Variables
  #self#::Core.Compiler.Const(walk2, false)
  T::Int64
  x::Int64
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Nothing
1 ─       (x = 0)
│   %2  = (1:T)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%2))
│   %4  = (@_4 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = x::Int64
│   %11 = Core.tuple(-1, 1)::Core.Compiler.Const((-1, 1), false)
│   %12 = Main.rand(%11)::Int64
│         (x = %10 + %12)
│         (@_4 = Base.iterate(%2, %9))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return


In [27]:
@time run(10_000, 10_000)

  0.890711 seconds (5 allocations: 176 bytes)


10280.3692

In [15]:
@time run(10_000, 100_000)

  2.529660 seconds (5 allocations: 176 bytes)


20070.74244

In [18]:
function my_sum(N)
    s = 0
    for i in 1:N
        s += i/2    # s = s + i/2
    end
    return s
end

my_sum (generic function with 1 method)

In [16]:
@code_lowered walk(100)

CodeInfo(:(begin 
        nothing
        x = 100 # line 4:
        SSAValue(0) = (Main.colon)(1, T)
        #temp# = (Base.start)(SSAValue(0))
        6: 
        unless !((Base.done)(SSAValue(0), #temp#)) goto 21
        SSAValue(1) = (Base.next)(SSAValue(0), #temp#)
        i = (Core.getfield)(SSAValue(1), 1)
        #temp# = (Core.getfield)(SSAValue(1), 2) # line 6:
        unless (Main.rand)() < 0.5 goto 16 # line 7:
        x = x + 1
        goto 19
        16:  # line 9:
        x = x - 1
        19: 
        goto 6
        21:  # line 13:
        return x
    end))

In [17]:
@code_typed walk(100)

CodeInfo(:(begin 
        x = 100 # line 4:
        SSAValue(6) = (Base.select_value)((Base.sle_int)(1, T)::Bool, T, (Base.sub_int)(1, 1)::Int64)::Int64
        #temp#@_4 = 1
        5: 
        unless (Base.not_int)((#temp#@_4 === (Base.add_int)(SSAValue(6), 1)::Int64)::Bool)::Bool goto 55
        SSAValue(7) = #temp#@_4
        SSAValue(8) = (Base.add_int)(#temp#@_4, 1)::Int64
        #temp#@_4 = SSAValue(8) # line 6:
        $(Expr(:inbounds, false))
        # meta: location random.jl rand 282
        # meta: location random.jl rand 143
        # meta: location random.jl reserve_1 132
        unless ((Core.getfield)(Base.Random.GLOBAL_RNG, :idx)::Int64 === (Base.sext_int)(Int64, Base.Random.MTCacheLength)::Int64)::Bool goto 26
        # meta: location random.jl gen_rand 128
        SSAValue(3) = (Core.getfield)(Base.Random.GLOBAL_RNG, :state)::Base.dSFMT.DSFMT_state
        SSAValue(2) = (Core.getfield)(Base.Random.GLOBAL_RNG, :vals)::Array{Float64,1}
        $(Expr(:invoke, MethodI

In [19]:
my_sum(10)

27.5

In [20]:
@time my_sum(10_000)

  0.000297 seconds (30.00 k allocations: 468.906 KiB)


2.50025e7

In [21]:
@time my_sum(10_000)

  0.000278 seconds (30.00 k allocations: 468.906 KiB)


2.50025e7

In [22]:
function my_sum2(N)
    s = 0.0
    for i in 1:N
        s += i/2    # s = s + i/2
    end
    return s
end

my_sum2 (generic function with 1 method)

In [23]:
my_sum2(10)

27.5

In [24]:
@time my_sum2(10_000)

  0.000012 seconds (5 allocations: 176 bytes)


2.50025e7

In [25]:
@code_warntype my_sum(10)

Variables:
  #self# <optimized out>
  N::Int64
  i::Int64
  #temp#@_4::Int64
  s::Union{Float64, Int64}
  #temp#@_6::Core.MethodInstance
  #temp#@_7::Float64

Body:
  begin 
      s::Union{Float64, Int64} = 0 # line 3:
      SSAValue(2) = (Base.select_value)((Base.sle_int)(1, N::Int64)::Bool, N::Int64, (Base.sub_int)(1, 1)::Int64)::Int64
      #temp#@_4::Int64 = 1
      5: 
      unless (Base.not_int)((#temp#@_4::Int64 === (Base.add_int)(SSAValue(2), 1)::Int64)::Bool)::Bool goto 30
      SSAValue(3) = #temp#@_4::Int64
      SSAValue(4) = (Base.add_int)(#temp#@_4::Int64, 1)::Int64
      i::Int64 = SSAValue(3)
      #temp#@_4::Int64 = SSAValue(4) # line 4:
      unless (s::Union{Float64, Int64} isa Int64)::Bool goto 15
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Int64, ::Float64)
      goto 24
      15: 
      unless (s::Union{Float64, Int64} isa Float64)::Bool goto 19
      #temp#@_6::Core.MethodInstance = MethodInstance for +(::Float64, ::Float64)
      goto 24
      

In [26]:
using Traceur

INFO: Precompiling module ASTInterpreter2.


In [27]:
@trace my_sum(10)

(my_sum)(::Int64) at In[18]:2
  s is assigned as Int64 at line 2
  s is assigned as Float64 at line 4
  dynamic dispatch to s + (Base.div_float)((Base.sitofp)(Float64, i), (Base.sitofp)(Float64, 2)) at line 4
  returns Union{Float64, Int64}


27.5

In [28]:
@trace my_sum2(10)

27.5

In [30]:
@code_llvm my_sum2(10)


define double @julia_my_sum2_63327(i64) #0 !dbg !5 {
top:
  %1 = icmp slt i64 %0, 1
  br i1 %1, label %L15, label %if.preheader

if.preheader:                                     ; preds = %top
  br label %if

if:                                               ; preds = %if.preheader, %if
  %s.03 = phi double [ %5, %if ], [ 0.000000e+00, %if.preheader ]
  %"#temp#.02" = phi i64 [ %2, %if ], [ 1, %if.preheader ]
  %2 = add i64 %"#temp#.02", 1
  %3 = sitofp i64 %"#temp#.02" to double
  %4 = fmul double %3, 5.000000e-01
  %5 = fadd double %s.03, %4
  %6 = icmp eq i64 %"#temp#.02", %0
  br i1 %6, label %L15.loopexit, label %if

L15.loopexit:                                     ; preds = %if
  br label %L15

L15:                                              ; preds = %L15.loopexit, %top
  %s.0.lcssa = phi double [ 0.000000e+00, %top ], [ %5, %L15.loopexit ]
  ret double %s.0.lcssa
}


In [32]:
@code_native my_sum2(10)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[22]
	pushq	%rbp
	movq	%rsp, %rbp
	xorpd	%xmm0, %xmm0
Source line: 3
	testq	%rdi, %rdi
	jle	L72
	xorpd	%xmm0, %xmm0
	xorl	%eax, %eax
	movabsq	$4795772760, %rcx       ## imm = 0x11DD9AF58
	movsd	(%rcx), %xmm1           ## xmm1 = mem[0],zero
	nopw	%cs:(%rax,%rax)
L48:
	incq	%rax
Source line: 4
	xorps	%xmm2, %xmm2
	cvtsi2sdq	%rax, %xmm2
	mulsd	%xmm1, %xmm2
	addsd	%xmm2, %xmm0
Source line: 3
	cmpq	%rax, %rdi
	jne	L48
Source line: 6
L72:
	popq	%rbp
	retq
	nopw	(%rax,%rax)
